In [1]:
#!pip install openai

In [2]:
from openai import OpenAI
from config import api_key
client = OpenAI(api_key=api_key)
client

In [3]:
uploaded_file = client.files.create(
    file=open("story.txt",'rb'),
    purpose='assistants'
)

In [4]:
uploaded_file

FileObject(id='file-FICi6duCgc7EjbVuG27wf6My', bytes=5299, created_at=1721070460, filename='story.txt', object='file', purpose='assistants', status='processed', status_details=None)

In [6]:
uploaded_file.id

'file-FICi6duCgc7EjbVuG27wf6My'

In [9]:
assistant = client.beta.assistants.create(
    name="Story helper",
    instructions="You are a motivator who answers the question based on the story file",
    tools=[{"type": "file_search"}],
    model="gpt-4o",
    #file_ids=[uploaded_file.id]
)

In [10]:
thread = client.beta.threads.create()
thread

Thread(id='thread_XpOSTS6VcWTGHtwyCRvHzVeI', created_at=1721070696, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [11]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Who is the hero of the story?"
)

In [12]:
message

Message(id='msg_92J7Bghi120IwinSuguyVxjH', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Who is the hero of the story?'), type='text')], created_at=1721070698, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_XpOSTS6VcWTGHtwyCRvHzVeI')

In [13]:
assistant

Assistant(id='asst_5v8C9qCCdKFq7pxEsbPO9s0b', created_at=1721070683, description=None, instructions='You are a motivator who answers the question based on the story file', metadata={}, model='gpt-4o', name='Story helper', object='assistant', tools=[FileSearchTool(type='file_search', file_search=None)], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=None, file_search=ToolResourcesFileSearch(vector_store_ids=[])), top_p=1.0)

In [24]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

In [25]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

In [31]:
run.status

'in_progress'

In [32]:
message

Message(id='msg_92J7Bghi120IwinSuguyVxjH', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Who is the hero of the story?'), type='text')], created_at=1721070698, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_XpOSTS6VcWTGHtwyCRvHzVeI')

In [33]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
messages

SyncCursorPage[Message](data=[Message(id='msg_BP8nGlbW8W2E9QqMjosL9y5g', assistant_id='asst_5v8C9qCCdKFq7pxEsbPO9s0b', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Please upload the story file so I can review it and determine who the hero of the story is.'), type='text')], created_at=1721070771, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_yerDJdRQJLvE2dhIK8LjAAno', status=None, thread_id='thread_XpOSTS6VcWTGHtwyCRvHzVeI'), Message(id='msg_BK0KJsc0d9rqwrJBXacxi03Q', assistant_id='asst_5v8C9qCCdKFq7pxEsbPO9s0b', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Please upload the story file so I can review it and determine who the hero of the story is.'), type='text')], created_at=1721070705, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_KgiQFKZyxxr5RozmKAPx5VGT',

In [34]:
while True:
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    if run.status=="completed":
        messages = client.beta.threads.messages.list(thread_id=thread.id)
        latest_message = messages.data[0]
        text = latest_message.content[0].text.value
        print(text)
        break;

Please upload the story file so I can review it and determine who the hero of the story is.
